In [1]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import numpy as np
import re
import time

In [2]:
def get_href(row):
    box_score_td = row.find_element(By.CSS_SELECTOR, 'td[data-stat="box_score_text"]')
    a = box_score_td.find_element(By.TAG_NAME, "a")
    href = a.get_attribute("href")
    return href

In [ ]:
years = [i for i in range(2012,2025)] #Getting from 2012 to this past season
months = ["october", "november", "december", "january", "february", "march", "april"]

df2 = pd.DataFrame(data=[], columns=["game_id", "Home Team", "Away Team", "Home Score", "Away Score", "url", "OT"])

#Loop through every possible url
driver = webdriver.Chrome()
game_id = 0
for year in years:
    #Start for one year.
    for month in months:
        url = f"https://www.basketball-reference.com/leagues/NBA_{year}_games-{month}.html"

        driver.get(url)
        try:
            header = driver.find_element(By.TAG_NAME, "h1")
            if "Page Not Found" in header.text:
                continue
        except:
            #h1 not found at all — hopefully a valid page
            pass
        time.sleep(1)

        try:
            schedule_table = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.ID, "schedule"))
                )
            rows = schedule_table.find_elements(By.TAG_NAME, "tr")
            for row in rows:
                data = row.find_elements(By.TAG_NAME, "td")
                if (len(data) > 7): #TBH there's no reason it shouldn't be like 10, but we only need the first few
                    away_team = data[1].text
                    home_team = data[3].text
                    away_score = data[2].text
                    home_score = data[4].text
                    ot = data[6].text
                    boxscore = get_href(row)
                    df2.loc[len(df2)] = [game_id, home_team, away_team, home_score, away_score, boxscore, ot]
                    game_id += 1 #Move on to the next game ID
        except TimeoutException:
            print(f"skipped {year} {month}")


        





In [45]:
df2.to_csv("../data/ScheduleData.csv", mode='a', header=True)